# Test Gaussian process.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

os.chdir("..")

In [46]:
import pandas as pd

from src import load, munge
from src.models import gp

## Load data.

In [4]:
matches = load.all_matches_df()

In [5]:
matches

,dire_name,dire_nicknames,dire_players,dire_valveId,duration,league_id,league_name,radiantVictory,radiant_name,radiant_nicknames,radiant_players,radiant_valveId,seriesId,startDate,startTimestamp
matchId,,,,,,,,,,,,,,,
4904311356,Forward Gaming,"[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,1964,10749,The International 2019,False,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,348164,2019-07-15 01:30:53,1563154253000
4904261164,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,1901,10749,The International 2019,True,Forward Gaming,"[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,348164,2019-07-15 00:19:24,1563149964000
4904218149,Forward Gaming,"[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,1956,10749,The International 2019,False,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,348164,2019-07-14 23:14:58,1563146098000
4904171017,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,2034,10749,The International 2019,False,Forward Gaming,"[pieliedie, Sneyking, MSS, YawaR, CCnC]","[6922000, 10366616, 86726887, 108452107, 22166...",6214538,348164,2019-07-14 22:09:50,1563142190000
4904062522,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,1817,10749,The International 2019,False,beastcoast,"[Brax, ixmike88, Ryoya, M-GOD, MoOz]","[31818853, 86715129, 115141430, 131706718, 349...",7079109,348080,2019-07-14 20:18:11,1563135491000
4903974414,J.Storm,"[Moo, Resolut1on, Fear, 1437, nine]","[84429681, 86725175, 87177591, 87196890, 94786...",6288801,2495,10749,The International 2019,False,beastcoast,"[Brax, ixmike88, Ryoya, M-GOD, MoOz]","[31818853, 86715129, 115141430, 131706718, 349...",7079109,348080,2019-07-14 19:02:07,1563130927000
4903679968,The Final Tribe,"[Handsken, Xibbe, Era, Frost, Chessie]","[18180970, 50580004, 100317750, 169359249, 172...",5059375,2521,10749,The International 2019,True,Chaos Esports Club,"[MATUMBAMAN, MISERY, MiLAN, vtFaded, KheZu]","[72312627, 87382579, 98172857, 142139318, 1690...",7203342,347944,2019-07-14 15:37:48,1563118668000
4903566049,Chaos Esports Club,"[MATUMBAMAN, MISERY, MiLAN, vtFaded, KheZu]","[72312627, 87382579, 98172857, 142139318, 1690...",7203342,1772,10749,The International 2019,False,The Final Tribe,"[Handsken, Xibbe, Era, Frost, Chessie]","[18180970, 50580004, 100317750, 169359249, 172...",5059375,347944,2019-07-14 14:41:46,1563115306000
4903441151,The Final Tribe,"[Handsken, Xibbe, Era, Frost, Chessie]","[18180970, 50580004, 100317750, 169359249, 172...",5059375,1460,10749,The International 2019,True,Chaos Esports Club,"[MATUMBAMAN, MISERY, MiLAN, vtFaded, KheZu]","[72312627, 87382579, 98172857, 142139318, 1690...",7203342,347944,2019-07-14 13:51:03,1563112263000


### Tangent: Radiant vs Dire win rate.

In [59]:
matches.radiantVictory.value_counts(True)

True     0.544
False    0.456
Name: radiantVictory, dtype: float64

In [39]:
def radiant_win_rate(s):
    return sum(s) / len(s)
def dire_win_rate(s):
    return 1 - radiant_win_rate(s)

In [58]:
temp = pd.concat([
    matches.groupby("radiant_name").agg({"radiantVictory": radiant_win_rate}),
    matches.groupby("dire_name").agg({"radiantVictory": dire_win_rate})],
    1,
    sort=True
)
temp.columns = ["radiant", "dire"]
temp.sort_values("radiant", ascending=False)

,radiant,dire
Team Freedom Dota 2,1.000000,1.000000
DreamEaters,1.000000,0.000000
CoM18Anji,1.000000,0.000000
DeathBringer Gaming,1.000000,0.000000
ReckoninG eSports,1.000000,NaN
Anticoagulation,1.000000,0.000000
Team Odd,1.000000,NaN
New Guys,0.900000,0.400000
Butterfly Effec,0.833333,0.750000
TeamJeKich,0.833333,0.250000


In [7]:
players_mat = munge.make_match_players_matrix(matches.dire_players,
                                              matches.radiant_players)

### Compute number of games by player.

In [19]:
game_count_by_player = abs(players_mat).sum()
game_count_by_player.value_counts().sort_index()

1.0      101
2.0       79
3.0       44
4.0       53
5.0       19
6.0       34
7.0       44
8.0       24
9.0       17
10.0      34
11.0      16
12.0      19
13.0      18
14.0      10
15.0      10
16.0      13
17.0       2
18.0      12
19.0      12
20.0      10
21.0      12
22.0       3
23.0       9
24.0      12
25.0       3
26.0       5
27.0       3
28.0       2
29.0       7
30.0       3
        ... 
236.0      1
237.0      3
238.0      1
241.0      1
242.0      1
243.0      1
248.0      1
251.0      3
252.0      1
256.0      1
257.0      1
258.0      1
259.0      4
262.0      2
268.0      1
272.0      2
273.0      1
274.0      2
279.0      3
281.0      2
282.0      4
287.0      4
288.0      1
294.0      1
300.0      2
302.0      1
312.0      2
316.0      1
333.0      1
349.0      1
Length: 196, dtype: int64

### Only keep players with more than 20 games, and discard matches as needed.

In [25]:
players_f_idx = game_count_by_player >= 20
players_mat_f = players_mat.loc[:, players_f_idx]
players_mat_f.shape

(5000, 414)

In [26]:
matches_f_idx = abs(players_mat_f).sum(1) == 10
players_mat_f = players_mat_f.loc[matches_f_idx]
players_mat_f.shape

(3742, 414)

## Finally, try the class.

In [ ]:
two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000
gp = gp.SkillsGP(players_mat_f.values, matches.startTimestamp[matches_f_idx],
                 matches.radiantVictory, players_mat_f.columns.values,
                 "exponential", {"scale": two_years_in_ms})